In [24]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import asyncio
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
app = Nominatim(user_agent="ny_restaurant")
geolocator = RateLimiter(app.geocode, min_delay_seconds=5, max_retries=5)
from tqdm import tqdm

tqdm.pandas()

from geopy.exc import GeocoderTimedOut


In [14]:
df = pd.read_csv("/Users/Chris/Desktop/COOP/DOHMH_New_York_City_Restaurant_Inspection_Results.csv")

In [15]:
#Note: some of the address have string length of 0 meaning we should check if the longitude/latitude data is there.
#Otherwise, remove the data point there
#Vice versa: If longitude/latitutde is missing, take the address and obtain the longitude and latitutde

In [16]:
#Step 1: Get the full address and see what's up

In [17]:
df["ZIPCODE"] = df["ZIPCODE"].fillna(0).astype(int)

In [18]:
df.shape

(319818, 26)

In [19]:
df["Address"] =  df["BUILDING"] + " "  + df["STREET"] + ", New York, " + df["ZIPCODE"].astype(str)
df["PART ADDRESS"] = df["BUILDING"] + " " + df["STREET"]
df["Address"].head()

0    603 FORT WASHINGTON AVENUE, New York, 10040
1          216 EAST  170 STREET, New York, 10456
2           105A DYCKMAN STREET, New York, 10040
3               9215 149 STREET, New York, 11435
4           13311 CROSSBAY BLVD, New York, 11417
Name: Address, dtype: object

In [20]:
a = df[df["Latitude"] == 0] 
b = df[df["Latitude"].isna()]
remove_these = pd.concat([a,b])
series = remove_these["ZIPCODE"] == 0
remove_these = remove_these[~series]
remove_these["Address"].dropna()
testing = remove_these[["Address"]]

In [30]:
test = remove_these[["Address"]].dropna()
test

,Address
63,"1 INTREPID SQ, New York, 10019"
594,"94 WYTHE AVE, New York, 11249"
863,"20408 BASHAN DR, New York, 20147"
1222,"1139/1141 CLARKSON AVE, New York, 11212"
1567,"NKA PENN STATION A, New York, 10121"
...,...
309547,"NKA PENN STATION A, New York, 10121"
310024,"122 E 42ND ST, New York, 10168"
311658,"94 WYTHE AVE, New York, 11249"
314733,"47 S 5TH ST, New York, 11249"


In [22]:
test.to_csv("testing.csv", index=False)

In [23]:
tdf = pd.read_csv('testing.csv')
tdf['Unique ID'] = range(1,len(tdf)+1)
tdf['Street_address'] = tdf['Address'].apply(lambda x: x.split(',')[0])
tdf['City'] = tdf['Address'].apply(lambda x: x.split(',')[1])
tdf['State'] = np.nan
tdf['ZIP'] = tdf['Address'].apply(lambda x: x.split(',')[2])
tdf.drop('Address', axis=1).to_csv('testing2.csv',index=False)
tdf.columns


Index(['Address', 'Unique ID', 'Street_address', 'City', 'State', 'ZIP'], dtype='object')

In [24]:
!curl --form addressFile=@testing2.csv --form benchmark=2020 https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output geocoderesult.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5842    0     0  100  5842      0   4610  0:00:01  0:00:01 --:--:--  4610
100  5842    0     0  100  5842      0   2636  0:00:02  0:00:02 --:--:--  2636
100 22740  100 16898  100  5842   7123   2462  0:00:02  0:00:02 --:--:--  9590


In [33]:
geocoded_census = pd.read_csv('geocoderesult.csv', names = ['Unique ID','Address','MATCH','TYPE','Found Address','Coords','Something','Something2'])

geocoded_census.shape

(140, 8)

In [ ]:
geocoded_census["MATCH"].value_counts()
geocoded_census = geocoded_census[~geocoded_census["Coords"].isna()]
geocoded_census["Latitude"] = geocoded_census["Coords"].apply(lambda x: str(x).split(",")[0])
geocoded_census["Longitude"] = geocoded_census["Coords"].apply(lambda x: str(x).split(",")[1])

geocoded_census

In [15]:
geocoded_census.drop(columns=['MATCH', 'TYPE','Coords', 'Something', 'Something2'])
geocoded_census.head()
geocoded.apply(lambda x: x)

,Unique ID,Address,MATCH,TYPE,Found Address,Coords,Something,Something2,Latitude,Longitude
0,88,"122 E 42ND ST, New York, , 10168",Match,Non_Exact,"122 E 42ND ST, NEW YORK, NY, 10017","-73.97628490999995,40.75153467600006",59658687.0,R,-73.97628490999995,40.75153467600006
1,89,"47 S 5TH ST, New York, , 11249",Match,Non_Exact,"47 S 5TH ST, BROOKLYN, NY, 11249","-73.96664307199995,40.712232638000046",59075970.0,L,-73.96664307199995,40.712232638000046
2,110,"45 S 3RD ST, New York, , 11249",Match,Non_Exact,"45 S 3RD ST, BROOKLYN, NY, 11249","-73.96561957099993,40.71352219500005",650082911.0,L,-73.96561957099993,40.71352219500005
3,111,"45 S 3RD ST, New York, , 11249",Match,Non_Exact,"45 S 3RD ST, BROOKLYN, NY, 11249","-73.96561957099993,40.71352219500005",650082911.0,L,-73.96561957099993,40.71352219500005
4,112,"45 S 3RD ST, New York, , 11249",Match,Non_Exact,"45 S 3RD ST, BROOKLYN, NY, 11249","-73.96561957099993,40.71352219500005",650082911.0,L,-73.96561957099993,40.71352219500005


In [17]:
df = df.set_index("Address")
geocoded_census = geocoded_census.set_index("Address")
df = df.fillna(geocoded_census)

KeyError: "None of ['Address'] are in the columns"